In [1]:
import pybedtools as pb
import numpy as np
#import scipy
import pandas as pd
#import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

/home/sergio/miniconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/sergio/miniconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/sergio/miniconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## Load enhancers 

In [13]:
enh = pb.BedTool('tracks/Enhancers_ChromHMM.bed')
enh_df = enh.to_dataframe()

# Enh to genes table
enh2gn = pd.read_table('tracks/Enhancers_ChromHMM_enh2genes.txt', 
                        header=1, 
                        names=['enhancers','gene_name'])
# Gene to enh table
gn2enh = pd.read_table('tracks/Enhancers_ChromHMM_genes2enh.txt', 
                        header=1, 
                        names=['gene_name','enhancers'])

def enh_gene(gen,df):
    reg = df[df['gene_name']==gen]
    if len(reg)>0:
        dr = reg.iloc[0]['enhancers'].split(' ')
        enhl = [x for x in dr if 'enh' in x]
    else:
        enhl=[]  
    return enhl


def enh_l_gene(genl,df):
    enhll = list()
    for gen in genl:
        reg = df[df['gene_name']==gen]
        if len(reg)>0:
            dr = reg.iloc[0]['enhancers'].split(' ')
            enhl = [x for x in dr if 'enh' in x]
        else:
            enhl=[]
        enhll = enhll+enhl
    return list(set(enhll))


def gene_enh(enh,df):
    reg = df[df['enhancers']==enh]
    if len(reg)>0:
        ls = reg.iloc[0]['gene_name'].split(', ')
        gl=[tr.split(' (')[0] for tr in ls]
    else:
        gl=[]  
    return gl

def gene_l_enh(enll,df):
    genl = list()
    for enh in enll:
        reg = df[df['enhancers']==enh]
        if len(reg)>0:
            ls = reg.iloc[0]['gene_name'].split(', ')
            gl=[tr.split(' (')[0] for tr in ls]
        else:
            gl=[]
        genl = genl+gl
    return list(set(genl))

In [9]:
#gens = ['Tm9sf2', 'Tle3']
#enhs = ['enh_168', 'enh_1683', 'enh_268']
#enh_l_gene(gens,gn2enh)
#enh_l_gene(gene_l_enh(enhs,enh2gn),gn2enh)
#enh_df.head()

,chrom,start,end,name
0,chr1,6397000,6397400,enh_0
1,chr1,6442800,6443400,enh_1
2,chr1,6445000,6446000,enh_2
3,chr1,6456800,6458000,enh_3
4,chr1,7444800,7445800,enh_4


## Load peaks 

In [10]:
tlx = pb.BedTool('tracks/TLX3_TLX3_peaks.bed')
tlx_df =tlx.to_dataframe(names=['chr','start','end','name','score'])

#shr = 100
#top = tlx_df[tlx_df['score']>shr].sort_values('score', axis=0, ascending=False)
#tlx_top = pb.BedTool.from_dataframe(top)

In [37]:
#(enh+tlx).head()

## Load TSS

In [16]:
genes = pb.BedTool('tracks/annot_tracks/genes.bed')

### == TSS region
tss =  3000

In [31]:
def gen_wth_peaks(en,enh2gn,tlx,genes,tss=3000):
    gn_tss = genes.slop(b=tss, genome='mm9')
    gn_tss_tlx=gn_tss+tlx
    gn_tss_tlx_df = gn_tss_tlx.to_dataframe()
    
    gn_tlx_l =list(gn_tss_tlx_df['name']) 
    
    gl = gene_enh(en,enh2gn)
    
    return list(set(gl) & set(gn_tlx_l))

In [57]:
enh_tlx = enh+tlx
enh_tlx_df = enh_tlx.to_dataframe()

In [60]:
import warnings
warnings.filterwarnings('ignore')

enh_tlx_df['gentlx'] = enh_tlx_df.apply(lambda row: gen_wth_peaks(row['name'],enh2gn,tlx,genes), axis=1)



In [66]:
enh_tlx_df['num_gentlx'] = enh_tlx_df.apply(lambda row: len(row['gentlx']), axis=1)


In [72]:
enh_tlx_df_sort = enh_tlx_df[enh_tlx_df['num_gentlx']>0]
enh_tlx_df_sort.sort_values('num_gentlx', axis=0, ascending=False,inplace = True)

In [75]:
#enh_tlx_df_sort.head()
len(enh_tlx_df_sort)

3347

## Load mutations

In [ ]:
ftlx = 'tracks/WGS-WES/Germline/FERRIER_09_Germline.allchr.snpEff.p.SAL.SAL10_1.vcf'
var_b = pb.BedTool(ftlx)

In [ ]:
#allTLX_list =list(all_tss_gene['name'])
#allTLX_list.sort()
#len(allTLX_list)

In [ ]:
#a,b,c = plt.hist(np.log(tlx_df['score']), bins=100)
#a,b,c = plt.hist(tlx_df['score'], bins=100)

## Intersect enhancers and high score peaks

In [ ]:
#enh_tlx_top = enh+tlx_top 

In [ ]:
#enh_tlx_top.head()

## Load WGS variants

In [ ]:
path = 'tracks/WGS-WES/Germline/'
ftlx = 'tracks/WGS-WES/Germline/FERRIER_09_Germline.allchr.snpEff.p.SAL.SAL10_1.vcf'
var_b = pb.BedTool(ftlx)

## Find variants in enhancers with TLX3 peaks

In [ ]:
var_ehn_tlx = var_b+enh_tlx_top

In [ ]:
var_ehn_tlx_df = var_ehn_tlx.to_dataframe(names=['CHROM','POS','ID','REF','ALT','QUAL','FILTER','INFO','FORMAT','AC3812','AC3813','AC3814','AC3815'])

In [ ]:
#var_ehn_tlx_df.head(10)
len(var_ehn_tlx_df)

In [ ]:
# Enhancers Table manipulation
enh2gn = pd.read_table('tracks/Enhancers_ChromHMM_enh2genes.txt', 
                        header=1, 
                        names=['enhancers','gene_name'])

In [ ]:
enh_tlx_top_df = enh_tlx_top.to_dataframe()

In [ ]:
enh_nm = list(enh_tlx_top_df['name'])

In [ ]:
enh2gn_top = enh2gn[enh2gn['enhancers'].isin(enh_nm)]

In [ ]:
a = list()
for i in range(len(enh2gn_top )):
    ls = enh2gn_top['gene_name'].iloc[i].split(', ')
    a=a+[tr.split(' (')[0] for tr in ls]


In [ ]:
len(set(a))

# Enrichment

In [ ]:

import EnrichRLib as erl
# List of gene sets as above
gss = [ 
       'GO_Biological_Process_2018',
       'GO_Cellular_Component_2018',
       'GO_Molecular_Function_2018',
       'KEGG_2016',
       'Reactome_2016'
       ]

In [ ]:
a = [x.upper() for x in a]

enr_a = erl.enrich_gs(a,gss)



In [ ]:
# --- Plot ---
enr_a.sort_values('p-Val', axis=0, inplace = True)
ds = enr_a.head(20)

f, ax = plt.subplots()
sns.barplot(y=ds.index,
            x='-log10(p-Val)',
            ax = ax, 
            color="Red", 
            data = ds)
ax.set_title('Genes regulated by enhancers with high Tlx3 peaks')

# Expression analysis

In [ ]:
import RNA_expression_processing as rn
from os.path import join

In [ ]:
tbl = pd.read_table(join('tracks', 'TLX3vsRAG-results_genesNames.txt'), index_col=0)
#tbl = tbl[(tbl.padj < 0.05)].dropna()

# === Pheno ==
A,B = 'TLX3','RAG'
classes = [A]*3+[B]*3


cols = ['Gene_name', 'TLX3.1_1','TLX3.1_5','TLX3.1_P','R2.RAG1W.RAG1','RAGS.RAGZ','RAGZ']

tbn = tbl[cols]
tbv = tbn.set_index(keys=tbn.columns[0])
tbv.index=tbv.index.str.upper()



In [ ]:

### == UP analysis

tbu = tbv.iloc[tbv.index.isin(a)]


In [ ]:
jj,kk,ll=rn.scatter_n(tbu, A, B, classes, n_top=3, ttl='') 

In [ ]:
ntop=40
gr = rn.cluster(tbu, A, B, classes, n_top=ntop)
gr.ax_heatmap.set_title('Cluster '+A+'/'+B)